# Quantum Forecasting for Energy Demand
## The Goal

It's a simple goal, I want to be AEMO at their own game!


https://github.com/ironfrown/qtsa_workshop

## Data Prep

I need 3 things:
1. AEMO's forecast for the next ? weeks at a point of time
2. Historical Energy Demand
3. Actuals over the forecasted period



## A Few Notes on the Architecture

